<a href="https://colab.research.google.com/github/JebaShazida/Network_Optimization_Model/blob/main/optimum_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp
import pulp
regions = ['North America', 'South America', 'Europe', 'Asia', 'Africa']
demand = {'North America': 12, 'South America': 8, 'Europe': 14, 'Asia': 16, 'Africa': 7}
plants = ['Plant 1', 'Plant 2', 'Plant 3', 'Plant 4', 'Plant 5']
capacities = {'Plant 1': {'low': 10, 'high': 20},
              'Plant 2': {'low': 10, 'high': 20},
              'Plant 3': {'low': 10, 'high': 20},
              'Plant 4': {'low': 10, 'high': 20},
              'Plant 5': {'low': 10, 'high': 20}}
fixed_costs = {'Plant 1': {'low': 6000, 'high': 9000},
               'Plant 2': {'low': 4500, 'high': 6750},
               'Plant 3': {'low': 6500, 'high': 9750},
               'Plant 4': {'low': 4100, 'high': 6150},
               'Plant 5': {'low': 4000, 'high': 6000}}
transportation_costs = {
    ('North America', 'Plant 1'): 81,
    ('North America', 'Plant 2'): 117,
    ('North America', 'Plant 3'): 102,
    ('North America', 'Plant 4'): 115,
    ('North America', 'Plant 5'): 142,
    ('South America', 'Plant 1'): 92,
    ('South America', 'Plant 2'): 77,
    ('South America', 'Plant 3'): 105,
    ('South America', 'Plant 4'): 125,
    ('South America', 'Plant 5'): 100,
    ('Europe', 'Plant 1'): 101,
    ('Europe', 'Plant 2'): 108,
    ('Europe', 'Plant 3'): 95,
    ('Europe', 'Plant 4'): 90,
    ('Europe', 'Plant 5'): 103,
    ('Asia', 'Plant 1'): 130,
    ('Asia', 'Plant 2'): 98,
    ('Asia', 'Plant 3'): 119,
    ('Asia', 'Plant 4'): 59,
    ('Asia', 'Plant 5'): 105,
    ('Africa', 'Plant 1'): 115,
    ('Africa', 'Plant 2'): 100,
    ('Africa', 'Plant 3'): 111,
    ('Africa', 'Plant 4'): 74,
    ('Africa', 'Plant 5'): 71,

}
model = pulp.LpProblem("Capacitated Plant Location Model", pulp.LpMinimize)
open_plant = pulp.LpVariable.dicts("Open", plants, cat=pulp.LpBinary)
transport = pulp.LpVariable.dicts("Transport", [(r, p) for r in regions for p in plants], lowBound=0, cat=pulp.LpContinuous)
model += pulp.lpSum(transportation_costs[(r, p)] * transport[(r, p)] for r in regions for p in plants) + \
         pulp.lpSum(fixed_costs[p][c] * open_plant[p] for p in plants for c in capacities[p])
for r in regions:
    model += pulp.lpSum(transport[(r, p)] for p in plants) == demand[r]
for p in plants:
    model += pulp.lpSum(transport[(r, p)] for r in regions) <= open_plant[p] * capacities[p]['low']
    model += pulp.lpSum(transport[(r, p)] for r in regions) <= (1 - open_plant[p]) * capacities[p]['high']
model.solve()
optimal_cost = pulp.value(model.objective)
print("Optimal cost:", optimal_cost)
print("Optimal solution:")
for p in plants:
    if pulp.value(open_plant[p]) > 0:
        print(f"{p} is open with capacity {capacities[p]['low']}")
        for r in regions:
            amount = pulp.value(transport[(r, p)])
            if amount > 0:
                print(f"   - {amount} units to {r}")


Optimal cost: 82516.00068180001
Optimal solution:
Plant 1 is open with capacity 10
   - 12.0 units to North America
Plant 2 is open with capacity 10
   - 8.0 units to South America
Plant 3 is open with capacity 10
   - 23.666667 units to Europe
Plant 4 is open with capacity 10
   - 6.6666667 units to Asia
Plant 5 is open with capacity 10
   - 10.666667 units to Asia
   - 7.0 units to Africa
